In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../../ukb-dementia-shap/")

import pickle
eth_map=dict({'British':'White',
 'Caribbean':'Other',
 'Any other white background':'White',
 'African':'African',
 'Other ethnic group':'Other',
 'Any other Asian background':'Asian',
 'Chinese':'Asian',
 'Indian':'Asian',
 'Irish':'White',
 'Pakistani':'Asian',
 'White and Black African':'African',
 'White and Asian':'Asian',
 'White and Black Caribbean':'Other',
 'White':'White',
 'Any other mixed background':'Other'})


from logic.data_processing.data_setup import *
from logic.analysis.analysis import AnalysisCharts
from logic.ml.classification_shap import IDEARs_funcs
from pandas.tseries.offsets import DateOffset

ds=data_setup()
an=AnalysisCharts()
dp=data_proc_main()
ml=IDEARs_funcs()

ml.path_figures='../figures/'

def prep_data(eids_in,eids_inc_depvar,eids_exc_depvar,depvar):
    mask=(df_model['eid'].isin(eids_in))&~(df_model['eid'].isin(eids_exc_depvar))
    df_out=df_model.loc[mask,]

    df_out[depvar]=0
    df_out.loc[df_out['eid'].isin(eids_inc_depvar),depvar]=1
    
    return df_out

def shap_multy(df,figname='test',depvar='pain',resize=0,resizeratio=20,meanshapmin=0.03,minrecs=3,rank=25,runs=10,barplots=1):
    
    feats_full=[]
    aucs_full=[]
    for i in range(runs):
        rets=ds.shapruns_new(df=df,run='test',\
    remwords='diabetes|H360|total_dis|Longstanding|illnesses|diseases',depvar=depvar,resize=resize,resizeratio=resizeratio,perc=True,barplots=barplots)

        feats=ml.shapgraphs_tuple(rets['shaps'],max_disp=30,figname='SHAP test',plot=False)
        feats['Variable_orig']=feats['Variable']
        feats['v2']=feats['Variable'].map(ds.varmap)
        feats.loc[pd.notnull(feats['v2']),'Variable']=feats['v2']
        feats_full.append(feats)
        aucs_full.append(rets['aucs'])
    #ml.boxplot_shap(df=feats_full,meanshapmin=meanshapmin,minrecs=minrecs,lab='all',figprint=True,figname=figname,format_file='.jpg',rank=rank,list=True)
    
    return feats_full,rets,aucs_full



In [ ]:
df_dis_date_test2=pd.read_parquet(dp.path+'df_dis_date_test2.parquet')
df_model=pd.read_parquet(dp.path+'df_all_final2022-04-13.parquet')
df_poly=pd.read_parquet('../../data/df_polyn_ukb.parquet')
df_pain=pd.read_parquet('../../data/df_pain_ukb.parquet')
df_brain_mri_only=pd.read_parquet('../../data/df_brain_mri.parquet')

#df_dis_date_test=pd.read_parquet(dp.path+'df_dis_date_test.parquet')
#df_model=pd.read_parquet(dp.path+'df_model_test_treat.parquet')
#df_model=pd.read_parquet(dp.path+'df_all_final'+dp.date_suff+'.parquet')



In [ ]:
df_brain_mri_only=pd.read_parquet('../../data/df_brain_mri.parquet')

In [ ]:
df_eth=pd.read_parquet('../../data/df_eth.parquet')

In [ ]:
df_pain['eid']=df_pain['eid'].astype(str)

In [ ]:
icd10_diabs_type1=ds.search_icd(strings='diabetes',second_string='insulin',
                                non_strings='family|screening|noninsulin|insipidus|pregnancy',string_pat=True)[0]
icd10_diabs_type2=ds.search_icd(strings='diabetes',second_string='noninsulin',
                                non_strings='family|screening|insipidus|pregnancy',string_pat=True)[0]

In [ ]:
#polyicd10=ds.return_eids(string=icd10_poly,icd10s=True)

In [ ]:
diabs_self_eids=list(df_poly.loc[df_poly['diabetes_diagnosed_by_doctor_f2443_0_0']==1,'eid'].astype(str))
diabpoly_self_eids=list(df_poly.loc[df_poly['ever_had_diabetic_neuropathy_f120079_0_0']==1,'eid'].astype(str))

In [ ]:
polys=ds.return_eids(df_dis_date_test2,string='polyneuropathy',icd10s=False,string_exc='xxxxx',years=2)
diabs=ds.return_eids(df_dis_date_test2,string='diabetes',string_exc='family|screening|insipidus|pregnancy') 
diabs_t1=ds.return_eids(df_dis_date_test2,string=icd10_diabs_type1,icd10s=True,string_exc='xxxxx',years=2)
diabs_t2=ds.return_eids(df_dis_date_test2,string=icd10_diabs_type2,icd10s=True,string_exc='xxxxx',years=2)

#### MRI

In [ ]:
df_brain_mri_only['diab_neur']=0
mask=df_brain_mri_only['eid'].isin(diabpoly_self_eids)
df_brain_mri_only.loc[mask,'diab_neur']=1
df_brain_mri_only['diab_neur'].sum()

In [ ]:
mask2=(df_brain_mri_only['eid'].isin(diabs_self_eids))
df_brain_mri_only2=df_brain_mri_only.loc[mask2,]

In [ ]:
#df_brain_mri_an=ds.varnorm(df=df_brain_mri_only,normvars=['age_when_attended_assessment_centre_f21003_0_0','sex_f31_0_0'],depvar='diab_neur')

feats_full_mri2=[]
aucs_full2=[]
for i in range(1):
    
    df_brain_mri_an=ds.varnorm(df=df_brain_mri_only2,normvars=['age_when_attended_assessment_centre_f21003_0_0','sex_f31_0_0'],depvar='diab_neur')


    feats_full_mri,rets_mri,aucs_full_mri=shap_multy(df=df_brain_mri_an,figname='MRI Diabetic Poly',depvar='diab_neur',
                              resize=0,resizeratio=10,meanshapmin=0.03,minrecs=3,rank=25,runs=10,barplots=0)
    feats_full_mri2=feats_full_mri2+feats_full_mri
    aucs_full2=aucs_full2+aucs_full_mri

In [ ]:
#df_brain_mri_only.to_parquet

In [ ]:
mri_feats=ml.boxplot_shap(df=feats_full_mri,meanshapmin=0.02,minrecs=1,lab='all',figprint=True,figname='../figures/SHAP_mri_diabetes_poly',
                format_file='.jpg',rank=25,list=True,ranknums=True,vars=['Variable','Variable_orig'])

In [ ]:
df_brain_mri_an=ds.varnorm(df=df_brain_mri_only,normvars=['age_when_attended_assessment_centre_f21003_0_0','sex_f31_0_0'],depvar='diab_neur')

feats_full_mri2=[]
aucs_full2=[]
for i in range(1):
    
    df_brain_mri_an=ds.varnorm(df=df_brain_mri_only,normvars=['age_when_attended_assessment_centre_f21003_0_0','sex_f31_0_0'],depvar='diab_neur')


    feats_full_mri,rets_mri,aucs_full_mri=shap_multy(df=df_brain_mri_an,figname='MRI Diabetic Poly',depvar='diab_neur',
                              resize=1,resizeratio=10,meanshapmin=0.03,minrecs=3,rank=25,runs=10,barplots=0)
    feats_full_mri2=feats_full_mri2+feats_full_mri
    aucs_full2=aucs_full2+aucs_full_mri

In [ ]:
mri_feats2=ml.boxplot_shap(df=feats_full_mri2,meanshapmin=0.02,minrecs=1,lab='all',figprint=True,figname='../figures/SHAP_mri_diabetes_poly',
                format_file='.jpg',rank=25,list=True,ranknums=True,vars=['Variable','Variable_orig'])

In [ ]:
othcols=['eid','age_when_attended_assessment_centre_f21003_0_0', 'sex_f31_0_0',
       'diab_neur']
compvars_mri_full=[c for c in df_brain_mri_only2.columns if c not in othcols]

In [ ]:
fn=pd.read_csv('../../data/fieldnames_full.csv')
map_col_notes=dict(zip(fn['col.name'],fn['Notes']))
mri_feats['Variable description']=mri_feats['Variable_orig'].map(map_col_notes)
mri_feats.to_csv('../data/mri_topfeats_desc.csv')

In [ ]:
compvars_mri=list(mri_feats['Variable_orig'])
feats_mri_orig=ds.runplots_static(df=df_brain_mri_only2,depvar='diab_neur',
                             fig_name='../figures/mri_diab_neur',perc=True,
                                  compvars=compvars_mri_full,savefig=False,pltshow=False)


In [ ]:
import statsmodels.stats.multitest as multi
new_ps=list(multi.multipletests(feats_mri_orig['p-value'])[1])

In [ ]:
feats_mri_orig['p-value']=feats_mri_orig['p-value'].astype(float)
new_val_dict=dict(zip([c for c in feats_mri_orig['p-value']],[c for c in new_ps]))

feats_mri_orig['adj p val']=feats_mri_orig['p-value'].map(new_val_dict)

In [ ]:
feats_mri_orig.to_csv('../data/mri_adjusted_pvals.csv')

In [ ]:
compvars_mri2=list(mri_feats2['Variable_orig'])
feats_mri_new=ds.runplots_static(df=df_brain_mri_only2,depvar='diab_neur',
                             fig_name='../figures/mri_diab_neur',perc=True,compvars=compvars_mri2,savefig=False)

In [ ]:
feats_mri_all=pd.concat([feats_mri_orig,feats_mri_new],axis=0)

In [ ]:
feats_mri_all.to_csv('../data/mri_pvals.csv')

In [ ]:
df_brain_mri_an=ds.varnorm(df=df_brain_mri_only2,normvars=['age_when_attended_assessment_centre_f21003_0_0'],
                           depvar='diab_neur')

compvars_mri=list(mri_feats['Variable_orig'])


feats_mri_an=ds.runplots_static(df=df_brain_mri_an,
        depvar='diab_neur',fig_name='../figures/mri_diab_neur_an',perc=True,compvars=compvars_mri,savefig=False)
feats_mri_an.to_csv('../data/feats_mri_an_diabetesonly_pvals.csv')

In [ ]:
feats_mri_an.reset_index().to_csv('../data/feats_mri_an_pvals.csv')

#### Diabetes

In [ ]:
%%time
mask=(df_model['eid'].isin(diabs['eids_inc_snap']))&~(df_model['eid'].isin(polys['eids_exc_pro']))
df_diab1a=df_model.loc[mask,]
df_diab1a['polyneuropathy']=0
df_diab1a.loc[df_diab1a['eid'].isin(polys['eids_inc_pro']),'polyneuropathy']=1

feats_full,rets,aucs_full=shap_multy(df=df_diab1a,figname='Polyneuropathy diabetes',depvar='polyneuropathy',
                          resize=0,resizeratio=20,meanshapmin=0.03,minrecs=10,rank=25,runs=50,barplots=0)

In [ ]:
ml.boxplot_shap(df=feats_full,meanshapmin=0.025,minrecs=10,lab='all',figprint=True,figname='../figures/SHAP_diabetes_poly',
                format_file='.jpg',rank=15,list=True,ranknums=True)

In [ ]:
feats_diab=ds.runplots_static(df=df_diab1a,depvar='polyneuropathy',fig_name='../figures/diabetes_poly',perc=True,compvars=ds.compvars)

#### Diabetes type 1 and 2

In [ ]:
eids_diabt1_snap=[e for e in diabs_t1['eids_inc_snap'] if e not in diabs_t2['eids_inc_snap']]
eids_diabt2_snap=[e for e in diabs_t2['eids_inc_snap'] if e not in diabs_t1['eids_inc_snap']]
eids_diabt12_snap=[e for e in diabs_t1['eids_inc_snap'] if e in diabs_t2['eids_inc_snap']]

In [ ]:
df_diab_t1=prep_data(eids_in=eids_diabt1_snap,eids_inc_depvar=polys['eids_inc_pro'],
                     eids_exc_depvar=polys['eids_exc_pro'],depvar='polyneuropathy')#diabs_t1['eids_inc_snap']

feats_full_t1,rets_t1,aucs_full_t1=shap_multy(df=df_diab_t1,figname='Polyneuropathy diabetes t1',depvar='polyneuropathy',
                          resize=0,resizeratio=20,meanshapmin=0.03,minrecs=10,rank=25,runs=50)


In [ ]:
ml.boxplot_shap(df=feats_full_t1,meanshapmin=0.025,minrecs=10,
                lab='all',figprint=True,figname='../figures/SHAP_diab_t1poly',format_file='.jpg',rank=15,list=True,ranknums=True)

In [ ]:
feats_diab_t1=ds.runplots_static(df=df_diab_t1,depvar='polyneuropathy',fig_name='../figures/diabetes_t1_poly',perc=True,compvars=ds.compvars)

In [ ]:
df_diab_t2=prep_data(eids_in=eids_diabt2_snap,eids_inc_depvar=polys['eids_inc_pro'],
                     eids_exc_depvar=polys['eids_exc_pro'],depvar='polyneuropathy')#diabs_t2['eids_inc_snap']

feats_full_t2,rets_t2,aucs_full_t2=shap_multy(df=df_diab_t2,figname='Polyneuropathy diabetes t2',depvar='polyneuropathy',
                          resize=0,resizeratio=20,meanshapmin=0.03,minrecs=10,rank=25,runs=50)


In [ ]:
ml.boxplot_shap(df=feats_full_t2,meanshapmin=0.025,minrecs=10,lab='all',figprint=True,
                figname='../figures/SHAP_diab_t2poly',format_file='.jpg',rank=15,list=True,ranknums=True)

In [ ]:
feats_diab_t2=ds.runplots_static(df=df_diab_t2,depvar='polyneuropathy',fig_name='../figures/diabetes_t2_poly',perc=True,compvars=ds.compvars)

#### Pain charts

In [ ]:
v='all_over_body_pain_in_last_three_months_and_rating_of_pain_f120022_0_0'
#v='feet_pain_in_last_three_months_and_rating_of_pain_f120033_0_0'
mask=(df_pain[v]>5)&(df_pain[v]<11)
pain_eids5=list(df_pain[mask]['eid'].unique())

In [ ]:
print('based on over 5 score for 3 months all body pain')
df_diabpoly_pain5=prep_data(eids_in=diabpoly_self_eids,eids_inc_depvar=pain_eids5,eids_exc_depvar=[],depvar='pain')
feats_full_p5,rets_p5,aucs_full_p5=shap_multy(df=df_diabpoly_pain5,figname='Pain over 5 score',depvar='pain',resize=0,
                            resizeratio=20,meanshapmin=0.03,minrecs=10,rank=25,runs=50,barplots=0)

In [ ]:
chk=pd.concat(feats_full_p5,axis=0)

In [ ]:
ml.boxplot_shap(df=feats_full_p5,meanshapmin=0.04,minrecs=20,lab='all',figprint=True,
                figname='../figures/pain_p5_SHAP',format_file='.jpg',rank=20,list=True,ranknums=True)

In [ ]:
feats_diab_p5=ds.runplots_static(df=df_diabpoly_pain5,depvar='pain',fig_name='../figures/diabet_poly_p5',perc=True,compvars=ds.compvars)

#### Putting all together

In [ ]:
feats_diab["Model"]='Polyneuropathy in Diabetes'
feats_diab_t1["Model"]='Polyneuropathy in Diabetes Type 1'
feats_diab_t2["Model"]='Polyneuropathy in Diabetes Type 2'
feats_diab_p5["Model"]='Pain All over body >5 in diabetic neuropathy'

feats_sum=pd.concat([feats_diab,feats_diab_t1,feats_diab_t2,feats_diab_p5],axis=0)

feats_sum.columns=[c[1]+' '+c[0] for c in feats_sum.columns]
feats_sum=feats_sum.reset_index()

feats_sum=feats_sum[ [' Model'] + [ col for col in feats_sum.columns if col != ' Model' ]]
feats_sum.to_csv('../data/summary_inflammation.csv')

In [ ]:
feats_sum

In [ ]:
aucs=[c[0] for c in aucs_full]+[c[0] for c in aucs_full_t2]+[c[0] for c in aucs_full_t1]+[c[0] for c in aucs_full_p5]+[c[0] for c in aucs_full_mri]
names=['Diabetes and Polyneuropathy' for c in aucs_full]+['Diabetes type 2 and Polyneuropathy' for c in aucs_full_t2]+\
['Diabetes type 1 and Polyneuropathy' for c in aucs_full_t1]\
+['Pain all over body >5 in diabetic polyneuropathy' for c in aucs_full_p5]+['MRI Age normalised diabetic neuropathy' for c in aucs_full_mri]

df_auc=pd.DataFrame({'model':names,'AUC':aucs})

print('AUCs for each model')
print(pd.DataFrame(df_auc.groupby(['model']).agg({'AUC':['mean','std']})).reset_index())

df_auc.to_csv('../data/diab_neur_aucs_wmri'+ml.date_run+'.csv')

from matplotlib.pyplot import figure
figure(figsize=(10, 10), dpi=200)
ax=sns.boxplot(data=df_auc,x='AUC',y='model',palette="Blues")
#ax.axes.set_title("Boxplot of AUC values for each experiment",fontsize=30)
ax.xaxis.label.set_visible(False)
ax.yaxis.label.set_visible(False)
              
plt.savefig("../figures/diab_neur_aucs_wmri"+ml.date_run+'.jpg', dpi=300,bbox_inches='tight')

plt.show()

#### Summary Table

In [ ]:
dfs=[df_diab1a,df_diab_t2,df_diab_t1,df_diabpoly_pain5,df_brain_mri_only]
depvars=['polyneuropathy','polyneuropathy','polyneuropathy','pain','diab_neur']
names=['Diabetes and Polyneuropathy','Diabetes type 1 and Polyneuropathy','Diabetes type 2 and Polyneuropathy',
       'Pain all over body >5 in diabetic polyneuropathy','MRI Age normalised diabetic neuropathy']

In [ ]:
df_eth['eid']=df_eth['eid'].astype(str)
eth_dic=dict(zip(df_eth['eid'],df_eth['ethnic_background_f21000_0_0']))

In [ ]:
df_age_full=pd.DataFrame([])
df_eth_full=pd.DataFrame([])
df_sex_full=pd.DataFrame([])
df_sex_eth_full=pd.DataFrame([])

for i,df in enumerate(dfs):
    df['ethnic_background_f21000_0_0']=df['eid'].map(eth_dic)
    df['ethnicity']=df['ethnic_background_f21000_0_0'].map(eth_map)
    df.loc[pd.isnull(df['ethnicity']),'ethnicity']='Unknown'
    df_sum=pd.DataFrame(df.groupby(depvars[i]).agg({'age_when_attended_assessment_centre_f21003_0_0':['mean','std']})).reset_index()
    df_sum['Model']=names[i]
    
    df_eth2=pd.DataFrame(df.groupby('ethnic_background_f21000_0_0').agg({depvars[i]:['sum','count']})).reset_index()
    df_eth2['Model']=names[i]
    
    df_sex=pd.DataFrame(df.groupby('sex_f31_0_0').agg({depvars[i]:['sum','count']})).reset_index()
    df_sex['Model']=names[i]
    
    df_sex_eth=pd.DataFrame(df.groupby(['sex_f31_0_0','ethnic_background_f21000_0_0']).agg({depvars[i]:['sum','count']})).reset_index()
    df_sex_eth['Model']=names[i]
    
    
    
    
    df_age_full=pd.concat([df_age_full,df_sum],axis=0)
    df_eth_full=pd.concat([df_eth_full,df_eth2],axis=0)
    df_sex_full=pd.concat([df_sex_full,df_sex],axis=0)
    df_sex_eth_full=pd.concat([df_sex_eth_full,df_sex_eth],axis=0)

dict_sum=dict({'Ages':df_age_full,'Ethnicities':df_eth_full,'Genders':df_sex_full,'Eth_Gender':df_sex_eth_full})


sum_file=open('../data/summary.pkl','wb')
pickle.dump(dict_sum,sum_file)
sum_file.close()


In [ ]:
dict_sum=pd.read_pickle('../data/summary.pkl')

In [ ]:
dict_sum['Eth_Gender']

In [ ]:
dict_sum['Eth_Gender'].to_csv('../data/ethgender_sum.csv')

In [ ]:
pd.DataFrame(df_diabpoly_pain5.groupby('ethnicity').agg({'pain':['sum','count']})).reset_index()